<a href="https://colab.research.google.com/github/aparnashenoy/ml/blob/master/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import the libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import the dataset

In [0]:
dataset=pd.read_csv('traindata.csv')


In [0]:
dataset.head()

,Unnamed: 0,X,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,1,1,84.0,98.0,37.06,118.5,77.00,58.5,18.0,24.0,0.50,7.39,40.0,97.0,57.0,18.0,78.0,8.3,106.0,0.9,1.4,124.0,1.8,2.0,3.4,4.1,0.9,4.3,30.2,10.4,32.4,10.8,250.0,181.0,83.14,0.0,1.0,0.0,-0.03,1.0,0.0
1,2,2,97.0,95.0,37.06,98.0,75.33,58.5,19.0,24.0,0.50,7.39,40.0,97.0,57.0,18.0,78.0,8.3,106.0,0.9,1.4,124.0,1.8,2.0,3.4,4.1,0.9,4.3,30.2,10.4,32.4,10.8,250.0,181.0,83.14,0.0,1.0,0.0,-0.03,2.0,0.0
2,3,3,89.0,99.0,37.06,122.0,86.00,58.5,22.0,24.0,0.50,7.39,40.0,97.0,57.0,18.0,78.0,8.3,106.0,0.9,1.4,124.0,1.8,2.0,3.4,4.1,0.9,4.3,30.2,10.4,32.4,10.8,250.0,181.0,83.14,0.0,1.0,0.0,-0.03,3.0,0.0
3,4,4,90.0,95.0,37.06,118.5,77.00,58.5,30.0,24.0,0.50,7.36,100.0,97.0,57.0,18.0,78.0,8.3,106.0,0.9,1.4,124.0,1.8,2.0,3.4,4.1,0.9,4.3,30.2,10.4,32.4,10.8,250.0,181.0,83.14,0.0,1.0,0.0,-0.03,4.0,0.0
4,5,5,103.0,88.5,37.06,122.0,91.33,58.5,24.5,24.0,0.28,7.39,40.0,97.0,57.0,18.0,78.0,8.3,106.0,0.9,1.4,124.0,1.8,2.0,3.4,4.1,0.9,4.3,30.2,10.4,32.4,10.8,250.0,181.0,83.14,0.0,1.0,0.0,-0.03,5.0,0.0


In [0]:
X=dataset.iloc[:,2:39].values

In [0]:
y=dataset.iloc[:,40]

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=0)

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)